# Visual Data Analysis of Fraudulent Transactions

Your CFO has also requested detailed trends data on specific card holders. Use the starter notebook to query your database and generate visualizations that supply the requested information as follows, then add your visualizations and observations to your markdown report.

In [132]:
# Initial imports
import pandas as pd
import calendar
import hvplot.pandas
from sqlalchemy import create_engine
%matplotlib inline

In [133]:
# Create a connection to the database
def postgres_connect_str(
    uname    = 'Mike',
    pass_env ='POSTGRES_PASSWORD' ,
    host     =  'pg-2e8191e-instructors-1f45.aivencloud.com',
    database = 'mike',
    port    = 18645,
    ):

    from dotenv import load_dotenv
    import os

    load_dotenv()
    password = os.getenv(pass_env)
    return f"postgresql://{uname}:{password}@{host}:{port}/{database}?sslmode=require"

postgres_connect_str()

engine = create_engine(postgres_connect_str())


## Data Analysis Question 1

The two most important customers of the firm may have been hacked. Verify if there are any fraudulent transactions in their history. For privacy reasons, you only know that their cardholder IDs are 2 and 18.

* Using hvPlot, create a line plot representing the time series of transactions over the course of the year for each cardholder separately. 

* Next, to better compare their patterns, create a single line plot that containins both card holders' trend data.  

* What difference do you observe between the consumption patterns? Does the difference suggest a fraudulent transaction? Explain your rationale in the markdown report.

In [134]:
# loading data for card holder 2 and 18 from the database
# Write the query
sql = """
        SELECT card_holder_id, date_trunc('day',date) as date, COUNT (*) as total FROM "Transactions" t
        INNER JOIN "Credit_card" c
        ON t."card_number" = c."card_number"
        WHERE card_holder_id = 2 OR card_holder_id = 18
        GROUP BY date_trunc('day',date), card_holder_id
        ORDER BY date_trunc('day',date)
        """
# Create a DataFrame from the query result. HINT: Use pd.read_sql(query, engine)
df = pd.read_sql_query(sql, engine)

df.index = pd.to_datetime(df["date"])

plot1 = df[df["card_holder_id"] == 18][["total","date"]].hvplot(label= "Customer 18") 


plot2 = df[df["card_holder_id"] == 2][["total","date"]].hvplot(label= "Customer 2") 

plot1 * plot2



:Overlay
   .Curve.Customer_18 :Curve   [date]   (total)
   .Curve.Customer_2  :Curve   [date]   (total)

In [135]:
std2= df["total"].std()*2

len(df[df['total'] >= std2])

199

In [136]:
# Plot for cardholder 2
plot2 = df[df["card_holder_id"] == 2][["total","date"]].hvplot(label= "Customer 2") 
plot2

:Curve   [date]   (total)

In [137]:
# Plot for cardholder 18
plot1 = df[df["card_holder_id"] == 18][["total","date"]].hvplot(label= "Customer 18") 
plot1


:Curve   [date]   (total)

In [138]:
# Combined plot for card holders 2 and 18
plot1 * plot2


:Overlay
   .Curve.Customer_18 :Curve   [date]   (total)
   .Curve.Customer_2  :Curve   [date]   (total)

## Data Analysis Question 2

The CEO of the biggest customer of the firm suspects that someone has used her corporate credit card without authorization in the first quarter of 2018 to pay quite expensive restaurant bills. Again, for privacy reasons, you know only that the cardholder ID in question is 25.

* Using Plotly Express, create a box plot, representing the expenditure data from January 2018 to June 2018 for cardholder ID 25.

* Are there any outliers for cardholder ID 25? How many outliers are there per month?

* Do you notice any anomalies? Describe your observations and conclusions in your markdown report.

In [139]:
# loading data of daily transactions from jan to jun 2018 for card holder 25
# Write the query
query = """
         SELECT date, SUM(amount) FROM "Transactions" t
        INNER JOIN "Credit_card" c
        ON t."card_number" = c."card_number"
        WHERE card_holder_id = 25 AND date BETWEEN '01/01/2018' AND '06/01/2018'
        GROUP BY date
        """
# Create a DataFrame from the query result. HINT: Use pd.read_sql(query, engine)
df2  = pd.read_sql(query,engine)
df2.index = pd.to_datetime(df2.date)
df2.hvplot()

:Curve   [date]   (sum)

In [144]:
monthly_df = df2.copy()
monthly_df.rename(columns={"sum":"Sum"}, inplace=True)
monthly_df.groupby([monthly_df.index.month, monthly_df.index.day]).sum()

Sum
date date         
1    2        1.46
     5       10.74
     7        2.93
     10       1.39
     14      17.84
     16       1.65
     18      15.86
     21       2.22
     30    1177.00
     31       2.75
2    2       10.75
     5       10.81
     7        5.97
     12       3.69
     18      16.70
     23      14.90
     28       2.09
3    2       12.42
     5       16.58
     6     1334.00
     7        2.88
     9        2.04
     11      13.57
     12      14.83
     16       4.20
     17       2.56
     18      18.28
     31      21.04
4    1      102.62
     2       24.23
     8     1083.21
     9      269.00
     10      10.24
     18       7.39
     19       6.01
     20      20.03
     26      28.47
     29      16.50
5    6        1.10
     13    1046.00
     17      12.15
     19       2.27
     29       5.97

In [145]:
# loop to change the numeric month to month names

monthly_df["Month"] = monthly_df["date"].apply(lambda x: calendar.month_name[x.month])
monthly_df["Day"] = monthly_df["date"].apply(lambda x: x.day)
monthly_df[["Sum", "Month", "Day"]][monthly_df["Month"]=="January"]




,Sum,Month,Day
date,,,
2018-01-31 05:46:43,2.75,January,31
2018-01-14 05:02:22,17.84,January,14
2018-01-30 18:31:00,1177.00,January,30
2018-01-02 02:06:21,1.46,January,2
2018-01-10 00:25:40,1.39,January,10
2018-01-05 06:26:45,10.74,January,5
2018-01-07 14:57:23,2.93,January,7
2018-01-21 23:04:02,2.22,January,21
2018-01-18 12:41:06,15.86,January,18


In [146]:
#Outliers per month
std = monthly_df['Sum'].std()*2+ monthly_df['Sum'].mean()
std

710.2198076583716

In [148]:
#Outliers
monthly_df["outlier"] = monthly_df["Sum"].apply(lambda x: 0 if x <std else 1)
monthly_df

,date,Sum,Month,Day,outlier
date,,,,,
2018-01-31 05:46:43,2018-01-31 05:46:43,2.75,January,31,0
2018-03-18 12:29:39,2018-03-18 12:29:39,18.28,March,18,0
2018-04-01 21:08:23,2018-04-01 21:08:23,2.62,April,1,0
2018-01-14 05:02:22,2018-01-14 05:02:22,17.84,January,14,0
2018-04-26 23:09:51,2018-04-26 23:09:51,15.66,April,26,0
2018-04-08 18:03:55,2018-04-08 18:03:55,10.15,April,8,0
2018-01-30 18:31:00,2018-01-30 18:31:00,1177.00,January,30,1
2018-05-29 14:34:36,2018-05-29 14:34:36,5.97,May,29,0
2018-04-29 02:41:44,2018-04-29 02:41:44,16.50,April,29,0


In [149]:
# Creating the six box plots using plotly express
monthly_df[["outlier", "Month", "Day"]][monthly_df["Month"]=="January"].drop(columns=["Day"]).hvplot.bar()


:Bars   [date]   (outlier)